In [3]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openclean

Note: you may need to restart the kernel to use updated packages.


In [5]:
import gzip
import humanfriendly
import os
import pandas as pd
import numpy as np
import csv

In [7]:
from openclean.data.source.socrata import Socrata

In [8]:
dataset = Socrata().dataset('erm2-nwe9')

# By default, this example uses a small sample of the full dataset that
# is included in the 'data' subfolder within this repository.
#datafile = './data/erm2-nwe9.tsv.gz'

# Remove the comment for this line if you want to use the full dataset.
datafile = './erm2-nwe9.tsv.gz'


# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)


fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using '311 Service Requests from 2010 to Present' in file ./erm2-nwe9.tsv.gz of size 2.91 GB


In [9]:
# Due to the size of the full dataset file, we make use of openclean's
# stream operator to avoid having to load the dataset into main-memory.

from openclean.pipeline import stream

ds_full = stream(datafile)

In [26]:
# Count number of records in the datasets.

print(f'{ds_full.count():,} rows.')


# Print the first ten rows of the dataset to get a first
# idea of the content.

ds_full.head()

27,122,579 rows.


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,34969964,12/08/2016 08:30:00 PM,12/09/2016 12:00:00 PM,DSNY,BCC - Brooklyn North,Missed Collection (All Materials),1R Missed Recycling-All Materials,Sidewalk,11212,790 ROCKAWAY AVENUE,...,,,,,,,,40.660413187950624,-73.90838198159439,"(40.660413187950624, -73.90838198159439)"
1,34969965,12/09/2016 08:14:00 AM,12/12/2016 12:00:00 PM,DSNY,BCC - Staten Island,Missed Collection (All Materials),1 Missed Collection,Sidewalk,10305,60 ALEX CIRCLE,...,,,,,,,,40.59301365304129,-74.06519740642362,"(40.59301365304129, -74.06519740642362)"
2,34969967,12/09/2016 08:53:00 AM,12/09/2016 10:30:00 AM,DOT,Department of Transportation,Traffic Signal Condition,Controller,,11234,,...,,,,,,,,40.61629487915584,-73.92989923852197,"(40.61629487915584, -73.92989923852197)"
3,34969968,12/09/2016 03:00:00 PM,12/10/2016 12:50:00 PM,DOT,Department of Transportation,Traffic Signal Condition,LED Pedestrian Unit,,11208,,...,,,,,,,,40.674384368841444,-73.87895984217538,"(40.674384368841444, -73.87895984217538)"
4,34969970,12/09/2016 06:54:15 PM,12/09/2016 06:57:55 PM,DHS,Operations Unit - Department of Homeless Services,Homeless Person Assistance,N/A,Bridge/Underpass,11211,185 MEEKER AVENUE,...,,,,,,,,40.715112875333205,-73.95230106983622,"(40.715112875333205, -73.95230106983622)"
5,34969971,12/09/2016 01:45:49 PM,12/09/2016 01:46:13 PM,TLC,Taxi and Limousine Commission,Taxi Report,Driver Report,,11369,LA GUARDIA AIRPORT,...,,,La Guardia Airport,,,,,40.77442086598845,-73.87729410513894,"(40.77442086598845, -73.87729410513894)"
6,34969973,12/09/2016 04:20:46 PM,12/09/2016 08:04:48 PM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10466,1890 SCHIEFFELIN AVENUE,...,,,,,,,,40.883841018857204,-73.84479382008124,"(40.883841018857204, -73.84479382008124)"
7,34969974,12/09/2016 09:00:00 AM,12/10/2016 12:00:00 PM,DSNY,BCC - Queens East,Missed Collection (All Materials),1RG Missed Recycling Paper,Sidewalk,11365,69-02 174 STREET,...,,,,,,,,40.73251928729983,-73.79588686870332,"(40.73251928729983, -73.79588686870332)"
8,34969978,12/09/2016 12:24:00 PM,12/09/2016 12:24:00 PM,DSNY,BCC - Brooklyn South,Missed Collection (All Materials),2 Bulk-Missed Collection,Sidewalk,11215,258 12 STREET,...,,,,,,,,40.66782983373874,-73.98875690143416,"(40.66782983373874, -73.98875690143416)"
9,34969979,12/09/2016 12:36:00 PM,12/10/2016 12:00:00 PM,DSNY,BCC - Staten Island,Missed Collection (All Materials),1RG Missed Recycling Paper,Sidewalk,10314,37 VILLA NOVA STREET,...,,,,,,,,40.584913088439436,-74.15879574248395,"(40.584913088439436, -74.15879574248395)"


In [10]:
# Create a view on a subset of columns in the dataset.

COLUMNS = [
    'Unique Key',
    'Created Date',
    'Closed Date',
    'Agency',
    'Agency Name',
    'Complaint Type',
    'Descriptor',
    'Location Type',
    'Incident Zip',
    'Incident Address',
    'Street Name',
    'Cross Street 1',
    'Cross Street 2',
    'Intersection Street 1',
    'Intersection Street 2',
    'Address Type',
    'City',
    'Landmark',
    'Facility Type',
    'Status',
    'Due Date',
    'Resolution Description',
    'Resolution Action Updated Date',
    'Community Board',
    'BBL',
    'Borough',
    'X Coordinate (State Plane)',
    'Y Coordinate (State Plane)',
    'Open Data Channel Type',
    'Park Facility Name',
    'Park Borough',
    'Vehicle Type',
    'Taxi Company Borough',
    'Taxi Pick Up Location',
    'Bridge Highway Name',
    'Bridge Highway Direction',
    'Road Ramp',
    'Bridge Highway Segment',
    'Latitude',
    'Longitude',
    'Location'
]

ds = ds_full.select(columns=COLUMNS)

In [15]:
type(dataset)

openclean.data.source.socrata.SODADataset

# Data Profiling

In [16]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.profile(default_profiler=DefaultColumnProfiler)

profiles.stats()

/Users/vaibhavbavishi/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PLACE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/vaibhavbavishi/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname AVE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/vaibhavbavishi/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname ST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this wi

,total,empty,distinct,uniqueness,entropy
Unique Key,27122579,0,27122579,1.000000e+00,24.692991
Created Date,27122579,0,18740648,6.909611e-01,22.749977
Closed Date,27122579,756422,12529609,4.752156e-01,21.241846
Agency,27122579,0,32,1.179829e-06,2.947540
Agency Name,27122579,0,1933,7.126903e-05,3.334448
Complaint Type,27122579,0,473,1.743934e-05,5.921467
Descriptor,27122579,66876,1790,6.615980e-05,7.338095
Location Type,27122579,6270214,194,9.303501e-06,3.167461
Incident Zip,27122579,1424622,2826,1.099698e-04,7.213478
Incident Address,27122579,4436646,1578973,6.960141e-02,18.246697


In [13]:
# Print overview of profiling results.

profiles.stats()

,total,empty,distinct,uniqueness,entropy
Unique Key,1000,0,1000,1.000000,9.965784
Created Date,1000,0,996,0.996000,9.957029
Closed Date,1000,23,972,0.994882,9.921979
Agency,1000,0,18,0.018000,2.886762
Agency Name,1000,0,50,0.050000,3.224148
Complaint Type,1000,0,123,0.123000,5.829106
Descriptor,1000,6,266,0.267606,6.889494
Location Type,1000,226,45,0.058140,3.090241
Incident Zip,1000,59,161,0.171095,7.059472
Incident Address,1000,171,817,0.985525,9.666278


In [20]:

# Print the most frequent data type for each column.

print('Schema\n------')
for col in ds.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'Unique Key' (int)
  'Created Date' (date)
  'Closed Date' (date)
  'Agency' (str)
  'Agency Name' (str)
  'Complaint Type' (str)
  'Descriptor' (str)
  'Location Type' (str)
  'Incident Zip' (int)
  'Incident Address' (str)
  'Street Name' (str)
  'Cross Street 1' (str)
  'Cross Street 2' (str)
  'Intersection Street 1' (str)
  'Intersection Street 2' (str)
  'Address Type' (str)
  'City' (str)
  'Landmark' (str)
  'Facility Type' (str)
  'Status' (str)
  'Due Date' (date)
  'Resolution Description' (str)
  'Resolution Action Updated Date' (date)
  'Community Board' (str)
  'BBL' (int)
  'Borough' (str)
  'X Coordinate (State Plane)' (int)
  'Y Coordinate (State Plane)' (int)
  'Open Data Channel Type' (str)
  'Park Facility Name' (str)
  'Park Borough' (str)
  'Vehicle Type' (str)
  'Taxi Company Borough' (str)
  'Taxi Pick Up Location' (str)
  'Bridge Highway Name' (str)
  'Bridge Highway Direction' (str)
  'Road Ramp' (str)
  'Bridge Highway Segment' (str)
  'Latitu